# CARLINI WAGNER #

Ricordati di cambiare le cartelle del test set.

#### *Import* 

In [9]:
import os
import re
import numpy as np
from matplotlib import pyplot as plt
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from art.estimators.classification import PyTorchClassifier
import tensorflow as tf


# Import all L-distance based attacks
from art.attacks.evasion import CarliniL2Method, CarliniL0Method, CarliniLInfMethod

#### Inizializzazione

In [10]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True

fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath)

def load_image(filename):
    img = Image.open(filename)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns

classifier = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631) #This class implements a classifier with the PyTorch framework.

#### Caricamento di una sola immagine per i test

In [12]:
test_img = load_image("test_set_cropped/Antonio_Cassano_8_face_0.jpg")

print(test_img.shape)
print(test_img.size)
test_img = test_img.unsqueeze(0)
print(test_img.shape)
print(test_img.size)
test_img = test_img.numpy()
print(test_img.shape)
print(type(test_img))

torch.Size([3, 160, 160])
<built-in method size of Tensor object at 0x000002020B7E3A98>
torch.Size([1, 3, 160, 160])
<built-in method size of Tensor object at 0x000002020B7E3908>
(1, 3, 160, 160)
<class 'numpy.ndarray'>


### *Attacco* **NON TARGETED**

In [15]:
dataset_dir = "test_set_cropped_piccolo/" 
binary_search_steps = 2
confidence = 0.7
max_iter = [1,2,5,7]
learning_rate = [0.01,0.001]
initial_const = 1000

accuracy_for_eps = []
accuracy_for_max_iter = []
perturbation_for_eps = []
perturbation_for_max_iter = []
correct_predictions = 0
total_images = 0

print("Inizio Attacco CARLINI-WAGNER NON-TARGETED")

for i in range(len(max_iter)):
        print("**** Attacco con max_iter = {} **** ".format(max_iter[i]))
        for learning in learning_rate:   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter[i], learning_rate=learning, initial_const=initial_const, targeted=False)
            
            print("**** Attacco con learning rate = {} ****".format(learning))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image(person_path)
                    test_img = test_img.unsqueeze(0)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img)
                    model_predictions = classifier.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Calcolo la perturbazione applicata su ogni immagine e la salvo in un vettore.
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1

                    accuracy = correct_predictions/total_images
                    print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))  # Calcolo l'accuracy attuale ogni volta che classifico una nuova immagine
            

            if total_images != 0:    
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images   # In media quanta pertubazione è stata applicata su ogni immagine 
                    perturbation_for_eps.append(perturbazione_media)
                    print("----------- Perturbazione media aggiunta a tutte le immagini per learning: {} equivale a ---> {}% ----------------".format(learning,perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps.append(final_accuracy)
                print("----------- Accuracy sugli adversarial Sample per learning: {} equivale a ---> {}% ----------------".format(learning,final_accuracy))

            
        if True:#len(perturbation_for_eps) == len(eps_range):
            perturbazione_media_per_max_iter = sum(perturbation_for_eps)/len(learning_rate) # Ottengo la media delle pertubazioni medie ottenute su ogni learning rate per una determinata iterazione
            perturbation_for_max_iter.append(perturbazione_media_per_max_iter)
            print("----------- Perturbazione media con iterazione {} sugli adversarial Sample ---> {}\% ----------------".format(max_iter[i],perturbation_for_max_iter[i]))
            perturbation_for_eps = []
        if True:#len(accuracy_for_eps) == len(eps_range):
            accuracy_media_per_max_iter = sum(accuracy_for_eps)/len(learning_rate) 
            accuracy_for_max_iter.append(accuracy_media_per_max_iter)
            print("----------- Accuracy media con iterazione {} sugli adversarial Sample ---> {}\% ----------------".format(max_iter[i],accuracy_for_max_iter[i]))
            accuracy_for_eps = []
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")


Inizio Attacco CARLINI-WAGNER NON-TARGETED
**** Attacco con max_iter = 1 **** 
**** Attacco con learning rate = 0.01 ****


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


Etichetta corretta: Andrea_Pirlo
Predetto  Marco_Giallini con probabilità 11.06852912902832 e con perturbazione 0.012716415338218212
Adversarial Sample misclassificati correttamente attuale: 100.0%


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


Etichetta corretta: Antonio_Cassano
Predetto  Jürgen_Trittin con probabilità 11.546632766723633 e con perturbazione 0.015857163816690445
Adversarial Sample misclassificati correttamente attuale: 100.0%


...

C&W L_2: 100%|██████████| 1/1 [00:03<00:00,  3.14s/it]


Etichetta corretta: Christian_De_Sica
Predetto  Marco_Antonio_Solís con probabilità 10.686558723449707 e con perturbazione 0.0017758214380592108
Adversarial Sample misclassificati correttamente attuale: 100.0%


C&W L_2: 100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


Etichetta corretta: Christopher_Nolan
Predetto  Franck_Dubosc con probabilità 12.482763290405273 e con perturbazione 0.0017528787720948458
Adversarial Sample misclassificati correttamente attuale: 100.0%


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

### *Attacco* **TARGETED**

In [14]:
dataset_dir = "test_set_cropped_piccolo/" 
binary_search_steps = 2
confidence = 0.7
max_iter = [1]
learning_rate = [0.01]
initial_const = 1000

accuracy_misclassified_for_eps = []
accuracy_misclassified_for_max_iter = []
accuracy_for_eps = []
accuracy_for_max_iter = []
perturbation_for_eps = []
perturbation_for_max_iter = []
correct_misclassified = 0
total_images = 0

target_class = 10
etichetta_target = LABELS[0]
print("ETICHETTA TARGET: ", LABELS[10])

targeted_labels = target_class*np.ones(LABELS.size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes = 8631)
test_images_adv = attack.generate(test_img, one_hot_targeted_labels)


print("Inizio Attacco CARLINI-WAGNER TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for learning in learning_rate:   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = CarliniL2Method(classifier=classifier, binary_search_steps=binary_search_steps, confidence=confidence, max_iter=max_iter[i], learning_rate=learning, initial_const=initial_const, targeted=False)
            
            print("**** Attacco con learning rate = {} ****".format(learning))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image(person_path)
                    test_img = test_img.unsqueeze(0)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
                    model_predictions = classifier.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1
                        
                    if etichetta_target in predicted_label:  
                        correct_misclassified = correct_misclassified+1   #Se il modello predice l'etichetta target allora è correttamente misclassificato

                    accuracy_misclassified = correct_misclassified/total_images
                    print("Adversarial Sample misclassificati correttamente attualmente: {}%".format((accuracy_misclassified)))
                    print("Accuracy attuale: {}%".format((correct_predictions/total_images)*100))
            

            if total_images != 0:    
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    
                    perturbation_for_eps.append(perturbazione_media)
                    print("----------- Perturbazione media aggiunta a tutte le immagini per learning: {} equivale a {}% ----------------".format(learning,perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps.append(final_accuracy)
                print("----------- Accuracy sugli adversarial Sample per learning: {} equivale a {}% ----------------".format(learning,final_accuracy))
                
                accuracy_misclassified = correct_misclassified/total_images
                accuracy_misclassified_for_eps.append(accuracy_misclassified)
                print("----------- Adversarial Sample misclassificati correttamente: {}% -----------".format((accuracy_misclassified)))

           
        if True:#len(perturbation_for_eps) == len(eps_range):
            perturbazione_media_per_max_iter = sum(perturbation_for_eps)/len(learning_rate) 
            perturbation_for_max_iter.append(perturbazione_media_per_max_iter)
            print("----------- Perturbazione media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],perturbation_for_max_iter[i]))
            perturbation_for_eps = []
        if True:#len(accuracy_for_eps) == len(eps_range):
            accuracy_media_per_max_iter = sum(accuracy_for_eps)/len(learning_rate) 
            accuracy_for_max_iter.append(accuracy_media_per_max_iter)
            print("----------- Accuracy media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_for_max_iter[i]))
            accuracy_for_eps = []
        if True:
            accuracy__misclassified_media_per_max_iter = sum(accuracy_misclassified_for_eps)/len(learning_rate) 
            accuracy_misclassified_for_max_iter.append(accuracy__misclassified_media_per_max_iter)
            print("----------- Accuracy MISCLASSIFIED media con iterazione {} sugli adversarial Sample: {}\% ----------------".format(max_iter[i],accuracy_misclassified_for_max_iter[i]))
            accuracy_misclassified_for_eps = []


        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")



ETICHETTA TARGET:   Aaron_Hernandez


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]


Inizio Attacco CARLINI-WAGNER TARGETED
**** Attacco con max_iter 1 **** 
**** Attacco con learning rate = 0.01 ****


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]


Etichetta corretta: Andrea_Pirlo
Predetto  Andrea_Pirlo con probabilità 12.725248336791992 e con perturbazione 0.0
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 100.0%


...

Etichetta corretta: Christopher_Nolan
Predetto  Christopher_Nolan con probabilità 12.074438095092773 e con perturbazione 0.0
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 87.5%


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]


Etichetta corretta: Cristiano_Ronaldo
Predetto  Cristiano_Ronaldo con probabilità 12.945648193359375 e con perturbazione 0.0
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 88.88888888888889%


C&W L_2: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


Etichetta corretta: Diego_Maradona
Predetto  Diego_Maradona con probabilità 12.622475624084473 e con perturbazione 0.0
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 90.0%
----------- Perturbazione media aggiunta a tutte le immagini per learning: 0.01 equivale a 0.0% ----------------
----------- Accuracy sugli adversarial Sample per learning: 0.01 equivale a 0.9% ----------------
----------- Adversarial Sample misclassificati correttamente: 0.0% -----------
----------- Perturbazione media con iterazione 1 sugli adversarial Sample: 0.0\% ----------------
----------- Accuracy media con iterazione 1 sugli adversarial Sample: 0.9\% ----------------
----------- Accuracy MISCLASSIFIED media con iterazione 1 sugli adversarial Sample: 0.0\% ----------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
